# Summary Statistics

In [1]:
import sqlite3

import httpx
import pandas as pd

from litreview import ClinicalTrials

# Make pandas display numbers in non-scientific notation
pd.set_option("display.float_format", lambda x: f"{x:0.1f}")

## Retrieve Integer Fields

In [2]:
API_BASE = "https://clinicaltrials.gov/api/v2"
API_FIELD_VALUES = API_BASE + "/stats/field/values"

response = httpx.get(API_FIELD_VALUES, params={"types": "INTEGER|NUMBER"})
response.raise_for_status()
data = response.json()
integer_fields = [field["piece"] for field in data]
integer_fields

['DispFirstPostYear',
 'DispFirstSubmitYear',
 'EnrollmentCount',
 'EventGroupDeathsNumAffected',
 'EventGroupDeathsNumAtRisk',
 'EventGroupOtherNumAffected',
 'EventGroupOtherNumAtRisk',
 'EventGroupSeriousNumAffected',
 'EventGroupSeriousNumAtRisk',
 'LargeDocSize',
 'LastUpdatePostYear',
 'LastUpdateSubmitYear',
 'NumArmGroupInterventionNames',
 'NumArmGroups',
 'NumAvailIPDs',
 'NumBaselineCategories',
 'NumBaselineClasses',
 'NumBaselineDenoms',
 'NumBaselineGroups',
 'NumBaselineMeasureDenoms',
 'NumBaselineMeasurements',
 'NumBaselineMeasures',
 'NumCentralContacts',
 'NumCollaborators',
 'NumCollaboratorsPlusLead',
 'NumConditionAncestors',
 'NumConditionBrowseBranches',
 'NumConditionBrowseLeafs',
 'NumConditionMeshes',
 'NumConditions',
 'NumDesignWhoMaskeds',
 'NumEventGroups',
 'NumEvents',
 'NumFlowAchievements',
 'NumFlowDropWithdraws',
 'NumFlowGroups',
 'NumFlowMilestones',
 'NumFlowPeriods',
 'NumFlowReasons',
 'NumIPDSharingInfoTypes',
 'NumInterventionAncestors',
 'N

## Load Data from Database

In [5]:
db_connection = sqlite3.connect("../clinical_trials.db")
trials = ClinicalTrials(connection=db_connection, schema_directory="../files/schema.json")
trials.query("EnrollmentCount")
# trials.query(*integer_fields)

[(None,),
 (40,),
 (112,),
 (0,),
 (212,),
 (12,),
 (164,),
 (45,),
 (320,),
 (99,),
 (429,),
 (600,),
 (17,),
 (10,),
 (50,),
 (15,),
 (120,),
 (36,),
 (463,),
 (34,),
 (25,),
 (40,),
 (33,),
 (50,),
 (549,),
 (108,),
 (30,),
 (90,),
 (0,),
 (16,),
 (15,),
 (84,),
 (73,),
 (22,),
 (12,),
 (864,),
 (16,),
 (18,),
 (253,),
 (146,),
 (900,),
 (80,),
 (12,),
 (32,),
 (1086,),
 (45,),
 (180,),
 (150,),
 (964,),
 (10,),
 (150,),
 (39,),
 (232,),
 (36,),
 (149,),
 (120,),
 (150,),
 (45,),
 (24,),
 (20,),
 (300,),
 (249,),
 (190,),
 (459,),
 (500,),
 (340,),
 (328,),
 (198,),
 (40,),
 (195,),
 (30,),
 (50,),
 (119,),
 (85,),
 (39,),
 (293,),
 (30,),
 (247,),
 (20,),
 (20,),
 (100,),
 (448,),
 (23,),
 (125,),
 (8,),
 (24,),
 (10,),
 (10,),
 (270,),
 (470,),
 (9,),
 (20,),
 (131,),
 (85,),
 (28,),
 (421,),
 (50,),
 (12,),
 (25,),
 (50,),
 (216,),
 (250,),
 (51,),
 (102,),
 (500,),
 (16,),
 (34,),
 (80,),
 (250,),
 (98,),
 (150,),
 (200,),
 (100,),
 (10,),
 (120,),
 (50,),
 (0,),
 (500,),
 (140,